In [86]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import time

In [11]:
pd.read_csv('/Users/chenjiayi/Downloads/data/FinalDataset/Phishing_Infogain_test.csv')

,domain_token_count,avgdomaintokenlen,tld,domainlength,pathurlRatio,argDomanRatio,domainUrlRatio,pathDomainRatio,NumberofDotsinURL,CharacterContinuityRate,host_letter_count,LongestPathTokenLength,SymbolCount_Domain,Entropy_Domain,class
0,2,5.50,2,12,0.76,0.17,0.15,5.08,1,0.75,11,48,1,0.86,0.0
1,3,5.00,3,17,0.69,0.12,0.22,3.18,3,0.65,15,40,2,0.78,0.0
2,2,4.00,2,9,0.76,0.78,0.13,6.00,1,0.67,8,22,1,1.00,0.0
3,2,4.50,2,10,0.73,0.20,0.16,4.70,1,0.80,9,33,1,0.88,0.0
4,2,6.00,2,13,0.71,2.69,0.19,3.69,2,0.77,12,39,1,0.83,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15362,2,8.00,2,17,0.35,0.12,0.46,0.76,1,0.82,16,8,1,0.88,1.0
15363,3,9.00,3,29,0.03,0.07,0.78,0.03,2,0.59,27,0,2,0.81,1.0
15364,3,6.67,3,22,0.22,0.09,0.59,0.36,2,0.50,20,6,2,0.80,1.0
15365,2,8.00,2,17,0.35,0.12,0.46,0.76,1,0.82,16,6,1,0.79,1.0


In [3]:
def dataload(filename):
    df = pd.read_csv(filename)
    df = df.replace('benign', '1').replace('Defacement', '0').replace('?', '0')
    return df

In [4]:
def dataclean(file):
    x = file.drop(['class'], axis = 1)
    y = file['class']
    return x,y

In [76]:
def model_accuracy(data, label):
    X_train, X_test, y_train, y_test = train_test_split(data, label, test_size = 0.2, random_state = 20)
    rfr = RandomForestRegressor(random_state = 42)
    rfr.fit(X_train, y_train)
    return rfr.score(X_train, y_train)

In [54]:
def best(data,label):
    X_train, X_test, y_train, y_test = train_test_split(data, label, test_size = 0.2, random_state = 20)
    tree_param_grid = { 'min_samples_split': list(range(3,9,1)),'n_estimators':list(range(10,100,10))}
    grid = GridSearchCV(RandomForestRegressor(),param_grid=tree_param_grid, cv=5)
    grid.fit(X_train, y_train)
    return grid.best_params_, grid.best_score_

In [84]:
def main(filename): 
    start_time = time.time()
    data = dataload(filename)
    X, y = dataclean(data)
    accuracy = model_accuracy(X, y)
    m, score_1 = best(X,y)
    print("original accuracy is:", score_1)
    estimator = m['n_estimators']
    min_sample = m['min_samples_split']
    score_2 = RF(X, y, estimator, min_sample)
    end_time = time.time()
    print("after optimize, the accuracy is:", score_2)
    print("the accuracy is increased:", score_2 - score_1)
    return (end_time - start_time)

In [80]:
def RF(data, label, estimators, min_sample):
    data_train, data_test, label_train, label_test = train_test_split(data, label, train_size = 0.8, random_state = 42)
    rfr = RandomForestClassifier(n_estimators = estimators, random_state = 42, min_samples_split = min_sample)
    rfr.fit(data_train, label_train)
    label_predict = rfr.predict(data_test)
    label_predict = np.array(label_predict)
    score = accuracy_score(label_predict.reshape(-1,1), np.array(label_test))
    return score
    

In [87]:
res = main('/Users/chenjiayi/Downloads/data/FinalDataset/Phishing_Infogain_test.csv')

original accuracy is: 0.9253257320845609
after optimize, the accuracy is: 0.9785296031229668
the accuracy is increased: 0.05320387103840585
time costs is: None
